In [ ]:



import openai
import pandas as pd
from sqlalchemy import create_engine
import os

# Database Configuration
DATABASE = "longeval-web"
USER = "dis18"
HOST = "db"
PORT = "5432"
PASSWORD = "dis182425"

# --- WICHTIG: Ersetzen Sie dies durch Ihren tatsächlichen OpenAI API Key ---
# Sichern Sie Ihren API-Schlüssel, z.B. durch Umgebungsvariablen
OPENAI_API_KEY = "" # Platzhalter
if "sk-proj-" in OPENAI_API_KEY and len(OPENAI_API_KEY) > 70 : # Einfache Prüfung auf Platzhalterformat
    print("WARNUNG: Bitte ersetzen Sie den Platzhalter OPENAI_API_KEY durch Ihren tatsächlichen Schlüssel in Zeile 15.")
    # exit() # Erwägen Sie, das Skript hier zu beenden, wenn der Schlüssel ein Platzhalter ist
openai.api_key = OPENAI_API_KEY


# Output file configuration
output_directory = "Expanded"
output_filename = "Query_Expansion_Final.csv"
output_file = os.path.join(output_directory, output_filename)

# Ensure the output directory exists
os.makedirs(output_directory, exist_ok=True)


def get_infos_from_openai(query_text):
    """
    Ruft einen Zeitabhängigkeits-Score von OpenAI für den gegebenen Query-Text ab.
    """
    prompt = f"You have this Query. Give a score on how time dependant this Query:{query_text}.The Score is between 0 to 1. Don't answer with anything more than the Score."
    try:
        response = openai.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=10, # Score ist kurz, 10 Tokens sollten reichen
            temperature=0  # Für deterministische Antworten
        )
        tag = response.choices[0].message.content.strip()
        return tag
    except Exception as e:
        print(f"      Fehler beim Aufruf der OpenAI API für Anfrage '{query_text}': {e}")
        return "ERROR_API_CALL" # Gibt ein eindeutiges Fehler-Tag zurück

def main():
    try:
        engine = create_engine(f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}")
        
        db_query = '''
        SELECT DISTINCT "text_fr"
        FROM "Topic";
        '''
        with engine.connect() as connection:
            df_from_db = pd.read_sql(db_query, con=connection)
        
        print(f"{len(df_from_db)} eindeutige Anfragen erfolgreich aus der Datenbank geladen.")

    except Exception as e:
        print(f"Fehler bei der Datenbankverbindung oder beim Abrufen der initialen Daten: {e}")
        return

    processed_queries_set = set()
    output_file_exists_and_valid = False

    if os.path.exists(output_file):
        try:
            # Versuche, die existierende Datei zu lesen
            existing_df = pd.read_csv(output_file)
            if not existing_df.empty and 'text_fr' in existing_df.columns:
                # Stelle sicher, dass text_fr als String behandelt wird für den Set-Vergleich
                processed_queries_set = set(existing_df['text_fr'].astype(str).tolist())
                print(f"{len(processed_queries_set)} bereits verarbeitete Anfragen aus '{output_file}' geladen.")
                output_file_exists_and_valid = True
            else:
                print(f"'{output_file}' ist leer oder hat keine 'text_fr' Spalte. Wird als neue Datei behandelt.")
        except pd.errors.EmptyDataError:
            print(f"'{output_file}' ist leer. Wird als neue Datei behandelt.")
        except Exception as e:
            print(f"Fehler beim Lesen der existierenden CSV-Datei '{output_file}': {e}. Starte Verarbeitung neu für diese Datei.")
    
    # Filtere die Datenbank-Anfragen, um nur neue zu bearbeiten
    # Stelle sicher, dass text_fr aus der DB auch als String verglichen wird
    df_from_db['text_fr'] = df_from_db['text_fr'].astype(str)
    df_to_process = df_from_db[~df_from_db['text_fr'].isin(processed_queries_set)].copy()

    num_new_queries = len(df_to_process)
    num_skipped_queries = len(df_from_db) - num_new_queries

    if num_skipped_queries > 0:
        print(f"{num_skipped_queries} Anfragen waren bereits verarbeitet und werden übersprungen.")

    if num_new_queries == 0:
        print("Keine neuen Anfragen zu bearbeiten. Alle Datenbankeinträge sind bereits in der CSV-Datei vorhanden.")
        return

    # Schreibe den Header, wenn die Datei nicht existierte oder nicht valide war
    if not output_file_exists_and_valid:
        try:
            # 'w' Modus überschreibt die Datei oder erstellt sie neu
            with open(output_file, 'w', newline='', encoding='utf-8') as f:
                header_df = pd.DataFrame(columns=['text_fr', 'tag'])
                header_df.to_csv(f, index=False)
            print(f"CSV-Header in '{output_file}' geschrieben (Datei neu erstellt oder war leer/inkonsistent).")
        except IOError as e:
            print(f"Fehler beim Schreiben des Headers in '{output_file}': {e}")
            return
    
    print(f"\nStarte OpenAI-Verarbeitung für {num_new_queries} neue Anfragen...")

    # Iteriere nur über die neuen/zu verarbeitenden Anfragen
    for i, (_, row) in enumerate(df_to_process.iterrows()): # (_, row) da der Index von df_to_process nicht unbedingt bei 0 beginnt
        current_query = row['text_fr']
        
        # Nummerierte Print-Ausgabe für den aktuellen Durchlauf
        print(f"Verarbeite neue Anfrage {i + 1}/{num_new_queries}: \"{current_query}\"")
        
        tag = get_infos_from_openai(current_query)
        
        print(f"  -> Tag erhalten: \"{tag}\"")
        
        row_to_append = pd.DataFrame([{'text_fr': current_query, 'tag': tag}])
        
        # Hänge an die CSV-Datei an (header=False, da er schon existiert oder gerade geschrieben wurde)
        try:
            # 'a' Modus für Anfügen
            with open(output_file, 'a', newline='', encoding='utf-8') as f:
                row_to_append.to_csv(f, header=False, index=False)
        except IOError as e:
            print(f"      Fehler beim Schreiben der Daten für Anfrage '{current_query}' in '{output_file}': {e}")

    print(f"\nVerarbeitung abgeschlossen. {num_new_queries} neue Anfragen wurden verarbeitet und in '{output_file}' gespeichert/angefügt.")

if __name__ == "__main__":
    main()

In [7]:
import pandas as pd
from sqlalchemy import create_engine
import os

# Datenbankkonfiguration (wie in Ihren vorherigen Skripten)
DATABASE = "longeval-web"
USER = "dis18"
HOST = "db"  # Stellen Sie sicher, dass 'db' vom Skriptumfeld aufgelöst werden kann
PORT = "5432"
PASSWORD = "dis182425"  # Vorsicht mit hartcodierten Passwörtern

# Eingabedatei: Die CSV mit den eindeutigen text_fr und ihren Tags
TAGGED_QUERIES_CSV = "Expanded/Query_Expansion_Final.csv"

# Optionale Ausgabedatei für das zusammengeführte Ergebnis
OUTPUT_MERGED_CSV = "Expanded/Topics_With_TimeTags.csv"

def main():
    # 1. Stelle eine Verbindung zur Datenbank her
    try:
        engine_string = f"postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}"
        engine = create_engine(engine_string)
        print("Erfolgreich mit der Datenbank verbunden.")
    except Exception as e:
        print(f"Fehler bei der Datenbankverbindung: {e}")
        return

    # 2. Lade die CSV-Datei mit den eindeutigen Anfragen und ihren Tags
    if not os.path.exists(TAGGED_QUERIES_CSV):
        print(f"FEHLER: Die Datei mit den Tags '{TAGGED_QUERIES_CSV}' wurde nicht gefunden.")
        print("Bitte stellen Sie sicher, dass das vorherige Skript zur Tag-Erstellung erfolgreich ausgeführt wurde.")
        return
    
    try:
        df_tags = pd.read_csv(TAGGED_QUERIES_CSV)
        # Stelle sicher, dass 'text_fr' für ein konsistentes Merging als String behandelt wird
        df_tags['text_fr'] = df_tags['text_fr'].astype(str)
        # Entferne Duplikate in 'text_fr', falls wider Erwarten vorhanden (sollte schon eindeutig sein)
        df_tags.drop_duplicates(subset=['text_fr'], inplace=True)
        print(f"{len(df_tags)} eindeutige Anfragen mit Tags aus '{TAGGED_QUERIES_CSV}' geladen.")
    except Exception as e:
        print(f"Fehler beim Laden der Tag-Datei '{TAGGED_QUERIES_CSV}': {e}")
        return

    # 3. Lade alle Daten aus der "Topic"-Tabelle ("alles laden")
    # Annahme: "alles laden" bedeutet alle Spalten aus der Tabelle "Topic"
    # Falls Sie eine spezifischere Auswahl benötigen, passen Sie die SQL-Query an.
    query_all_topics = 'SELECT * FROM "Topic";'  # Lädt alle Spalten
    try:
        with engine.connect() as connection:
            df_all_topics = pd.read_sql(query_all_topics, con=connection)
        # Stelle auch hier sicher, dass 'text_fr' für das Merging als String behandelt wird
        if 'text_fr' in df_all_topics.columns:
            df_all_topics['text_fr'] = df_all_topics['text_fr'].astype(str)
        else:
            print("FEHLER: Die Spalte 'text_fr' wurde nicht in der 'Topic'-Tabelle gefunden.")
            return
            
        print(f"{len(df_all_topics)} Einträge aus der Tabelle 'Topic' geladen.")
    except Exception as e:
        print(f"Fehler beim Laden der Daten aus der Tabelle 'Topic': {e}")
        return

    # 4. Führe die DataFrames zusammen
    # Ein Left-Merge wird verwendet, um alle Zeilen aus df_all_topics beizubehalten
    # und die 'tag'-Spalte aus df_tags hinzuzufügen, wo 'text_fr' übereinstimmt.
    df_merged = pd.merge(df_all_topics, df_tags, on='text_fr', how='left')
    
    print(f"\nDaten erfolgreich zusammengeführt.")
    print(f"Das resultierende DataFrame hat {len(df_merged)} Zeilen und {len(df_merged.columns)} Spalten.")

    # 5. Zeige einige Informationen und eine Vorschau der zusammengeführten Daten
    print("\nVorschau der zusammengeführten Daten (die ersten 5 Zeilen):")
    print(df_merged.head())

    print("\nInformationen zum zusammengeführten DataFrame:")
    df_merged.info()

    # Überprüfe, ob es Einträge gab, denen kein Tag zugeordnet werden konnte
    # Dies sollte idealerweise 0 sein, wenn alle 'text_fr' aus 'Topic' in der Tag-Datei vorhanden waren.
    untagged_count = df_merged['tag'].isnull().sum()
    if untagged_count > 0:
        print(f"\nWARNUNG: {untagged_count} von {len(df_merged)} Einträgen in der 'Topic'-Tabelle konnten keinem Tag zugeordnet werden.")
        print("Dies kann passieren, wenn einige 'text_fr'-Werte aus der 'Topic'-Tabelle nicht in Ihrer '{TAGGED_QUERIES_CSV}'-Datei enthalten waren.")
        # print("Beispiele für nicht getaggte Anfragen (erste 5):")
        # print(df_merged[df_merged['tag'].isnull()].head())
    else:
        print("\nAlle Einträge aus der 'Topic'-Tabelle konnten erfolgreich mit Tags versehen werden.")

    # 6. Optional: Speichere das zusammengeführte DataFrame in einer neuen CSV-Datei
    try:
        # Stelle sicher, dass das Ausgabeverzeichnis existiert
        os.makedirs(os.path.dirname(OUTPUT_MERGED_CSV), exist_ok=True)
        df_merged.to_csv(OUTPUT_MERGED_CSV, index=False)
        print(f"\nDie zusammengeführten Daten wurden erfolgreich in '{OUTPUT_MERGED_CSV}' gespeichert.")
    except Exception as e:
        print(f"\nFehler beim Speichern der zusammengeführten Daten in '{OUTPUT_MERGED_CSV}': {e}")

if __name__ == "__main__":
    main()

Erfolgreich mit der Datenbank verbunden.
47053 eindeutige Anfragen mit Tags aus 'Expanded/Query_Expansion_Final.csv' geladen.
153891 Einträge aus der Tabelle 'Topic' geladen.

Daten erfolgreich zusammengeführt.
Das resultierende DataFrame hat 153891 Zeilen und 7 Spalten.

Vorschau der zusammengeführten Daten (die ersten 5 Zeilen):
                                     id queryid text_en  \
0  f72ba204-9502-4e2b-b5a3-886df465b726       3    None   
1  238e44f2-87f9-49fb-93d2-70d983da668d       4    None   
2  6e75448f-4832-4a59-acbf-e14722c3a48f       7    None   
3  dd3e414c-78fd-4df9-a4fc-56d2ad8950b8       8    None   
4  e25ebbf6-5250-4c29-b231-1dc1ede1683e       9    None   

                    text_fr sub_collection split  tag  
0      1ere guerre mondiale        2022-06  test  0.2  
1  1ere guerre mondiale ce2        2022-06  test  0.9  
2          3949 pole emploi        2022-06  test  0.5  
3  4 mariages 1 enterrement        2022-06  test  0.2  
4             4k video down     

In [6]:
import pandas as pd
import os

# --- Konfiguration ---
# Bitte geben Sie den Pfad zu der CSV-Datei an, die Sie bereinigen möchten.
# Zum Beispiel: "Expanded/Query_Expansion_Final.csv" oder "Expanded/Topics_With_TimeTags.csv"
INPUT_CSV_FILE = "Expanded/Query_Expansion_Final.csv"
# Die bereinigte Datei wird standardmäßig die Originaldatei überschreiben.
# Wenn Sie eine neue Datei erstellen möchten, ändern Sie den folgenden Namen:
OUTPUT_CSV_FILE = INPUT_CSV_FILE # Überschreibt die Originaldatei

# Name der Spalte, die die Tags enthält
TAG_COLUMN_NAME = 'tag'

def clean_time_tag(tag_value):
    """
    Bereinigt einen einzelnen Tag-Wert.
    Gibt eine Zahl zwischen 0.0 und 1.0 zurück oder 0.0, wenn der Wert ungültig ist.
    """
    try:
        # Versuche, den Wert in eine Fließkommazahl umzuwandeln
        numeric_tag = float(tag_value)
        # Überprüfe, ob die Zahl im gültigen Bereich (0 bis 1) liegt
        if 0.0 <= numeric_tag <= 1.0:
            return numeric_tag
        else:
            # Zahl ist außerhalb des Bereichs, setze auf 0.0
            return 0.0
    except (ValueError, TypeError):
        # Umwandlung fehlgeschlagen (z.B. Text wie "ERROR_API_CALL" oder der Wert ist None)
        # Setze auf 0.0
        return 0.0

def main():
    # Überprüfe, ob die Eingabedatei existiert
    if not os.path.exists(INPUT_CSV_FILE):
        print(f"FEHLER: Die Eingabedatei '{INPUT_CSV_FILE}' wurde nicht gefunden.")
        return

    print(f"Lese Datei: '{INPUT_CSV_FILE}'...")
    try:
        df = pd.read_csv(INPUT_CSV_FILE)
    except Exception as e:
        print(f"FEHLER beim Lesen der CSV-Datei: {e}")
        return

    # Überprüfe, ob die Tag-Spalte existiert
    if TAG_COLUMN_NAME not in df.columns:
        print(f"FEHLER: Die Spalte '{TAG_COLUMN_NAME}' wurde in der Datei '{INPUT_CSV_FILE}' nicht gefunden.")
        print(f"Vorhandene Spalten sind: {df.columns.tolist()}")
        return

    print(f"Bereinige die Spalte '{TAG_COLUMN_NAME}'...")

    # Zähle, wie viele Werte geändert werden
    original_tags = df[TAG_COLUMN_NAME].copy()
    
    # Wende die Bereinigungsfunktion auf jede Zelle in der Tag-Spalte an
    df[TAG_COLUMN_NAME] = df[TAG_COLUMN_NAME].apply(clean_time_tag)

    # Vergleiche die Originalwerte mit den neuen Werten, um die Anzahl der Änderungen zu ermitteln
    # Beachte: Der Vergleich von Fließkommazahlen kann heikel sein, aber hier vergleichen wir
    # das Ergebnis der Funktion (die 0.0 oder den Originalwert zurückgibt, wenn er gültig war)
    # mit dem Original. Eine genauere Methode wäre, die Anzahl der Rückgaben von 0.0 zu zählen,
    # wenn der Originalwert nicht 0.0 und gültig war.
    # Einfacher: Zähle, wo sich der Wert geändert hat, nachdem sichergestellt wurde, dass beide numerisch sind für den Vergleich.
    
    # Um die Anzahl der tatsächlich geänderten Werte zu ermitteln (die nicht schon 0.0 waren oder gültig):
    changed_count = 0
    for original, new in zip(original_tags, df[TAG_COLUMN_NAME]):
        # Prüfen, ob der neue Wert 0.0 ist UND der Originalwert nicht bereits ein gültiger Wert war, der zu 0.0 hätte werden können
        # oder ob der Originalwert ein ungültiger String war.
        try:
            orig_float = float(original)
            if not (0.0 <= orig_float <= 1.0) and new == 0.0: # War numerisch aber ungültig, wurde zu 0.0
                 changed_count +=1
            elif (0.0 <= orig_float <= 1.0) and orig_float != new : # War gültig, wurde aber trotzdem geändert (sollte nicht passieren mit clean_time_tag Logik)
                 changed_count +=1 # Sicherheitscheck
        except (ValueError, TypeError): # War nicht-numerisch (z.B. String) und wurde zu 0.0 (new == 0.0 ist implizit)
            if new == 0.0: # Überprüfen, ob es tatsächlich zu 0.0 wurde
                 changed_count +=1


    print(f"Bereinigung abgeschlossen. {changed_count} Tag(s) wurden auf 0.0 gesetzt oder angepasst.")
    
    # Zähle die Verteilung der neuen Tags (optional, zur Information)
    print("\nVerteilung der Werte in der bereinigten Spalte '%s':" % TAG_COLUMN_NAME)
    print(df[TAG_COLUMN_NAME].value_counts(dropna=False).sort_index())


    # Speichere das bereinigte DataFrame
    try:
        # Stelle sicher, dass das Ausgabeverzeichnis existiert, falls es Teil des Pfades ist
        output_dir = os.path.dirname(OUTPUT_CSV_FILE)
        if output_dir and not os.path.exists(output_dir):
            os.makedirs(output_dir)
            
        df.to_csv(OUTPUT_CSV_FILE, index=False)
        if INPUT_CSV_FILE == OUTPUT_CSV_FILE:
            print(f"\nDie bereinigten Daten wurden erfolgreich in '{OUTPUT_CSV_FILE}' gespeichert (Original überschrieben).")
        else:
            print(f"\nDie bereinigten Daten wurden erfolgreich in '{OUTPUT_CSV_FILE}' gespeichert.")
    except Exception as e:
        print(f"FEHLER beim Speichern der bereinigten CSV-Datei: {e}")

if __name__ == "__main__":
    main()

Lese Datei: 'Expanded/Query_Expansion_Final.csv'...
Bereinige die Spalte 'tag'...
Bereinigung abgeschlossen. 1 Tag(s) wurden auf 0.0 gesetzt oder angepasst.

Verteilung der Werte in der bereinigten Spalte 'tag':
tag
0.0      780
0.1     8471
0.2    23104
0.3     3321
0.5      716
0.7     2859
0.8     5819
0.9     1410
1.0      573
Name: count, dtype: int64

Die bereinigten Daten wurden erfolgreich in 'Expanded/Query_Expansion_Final.csv' gespeichert (Original überschrieben).
